In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import pickle
from textblob import TextBlob
from sklearn.metrics import  f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
from sklearn.feature_extraction import text
from tqdm import  tqdm
stop_words = text.ENGLISH_STOP_WORDS.union(["book"])
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [0]:
def format_txt(x):
  if x!='':
    x=[' '.join(x1.split()) for x1 in x]
    return x
  else:
    return x

def format_text(x):
  if x!='':
    x=' '.join(x.split())
    return x
  else:
    return x

def remove_html_tag(text):
    """Remove html tags from a string"""
    import re
    if len(text)>=0:
      clean = re.compile('<.*?>')
      text=[re.sub(clean, '', text1) for text1 in text]
      return text
    else: 
      return text

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    if text!='':
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    else:
      return text

In [0]:
df=pd.read_csv('/content/drive/My Drive/TF_QA_DATA_SQuAD_FORMAT/TF_QA_DATA_SQuAD_FORMAT.csv')
df.fillna('',inplace=True)

In [5]:
df.head(5)

,example_id,text,candidate_answers,question,pred_short_ans,pred_long_ans,pred_yes_no_ans,doc_url
0,5655493461695504401,Email marketing - Wikipedia <H1> Email marketi...,['<Table> <Tr> <Td> </Td> <Td> ( hide ) This a...,which is the most common use of opt-in e-mail ...,a newsletter sent to an advertising firm 's cu...,<P> A common example of permission marketing i...,,https://en.wikipedia.org//w/index.php?title=Em...
1,5328212470870865242,The Mother ( How I Met Your Mother ) - wikiped...,"['<Table> <Tr> <Th_colspan=""2""> Tracy McConnel...",how i.met your mother who is the mother,Tracy McConnell,"<P> Tracy McConnell , better known as `` The M...",,https://en.wikipedia.org//w/index.php?title=Th...
2,4435104480114867852,Human fertilization - wikipedia <H1> Human fer...,['<Table> <Tr> <Td> Part of a series on </Td> ...,what type of fertilisation takes place in humans,,<P> The process of fertilization involves a sp...,,https://en.wikipedia.org//w/index.php?title=Hu...
3,5289242154789678439,List of National Football League career quarte...,"[""<Table> <Tr> <Th> NFL lists </Th> </Tr> <Tr>...",who had the most wins in the nfl,Tom Brady,<P> Active quarterback Tom Brady holds the rec...,,https://en.wikipedia.org//w/index.php?title=Li...
4,5489863933082811018,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[""<Table> <Tr> <Td> </Td> <Td> This article 's...",what happened to the lost settlement of roanoke,,,,https://en.wikipedia.org//w/index.php?title=Ro...


In [6]:
document=list()
correct_txt=list()
possible_ans=list()
qs=list()
pred_txt=list()
for idx in tqdm(range(0,500)):

  corpus=format_text(remove_html_tags(df.loc[idx,'text']))
  correct_text=format_text(remove_html_tags(df.loc[idx,'pred_long_ans']))
  question=format_text(remove_html_tags(df.loc[idx,'question']))
  possible_answers=format_txt(remove_html_tag(eval(df.loc[idx,'candidate_answers'])))
 
  tfidf=TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words)
  tfidf.fit([corpus])
  score=list()
  q_tfidf=tfidf.transform([question]).todense()
  for idx, x1 in enumerate(possible_answers):
    poss_tfidf=tfidf.transform([x1]).todense()
    scr=1-spatial.distance.cosine(q_tfidf,poss_tfidf)
    score.append(scr)
  
  pred_ans=possible_answers[np.argmax(score)]

  document.append(corpus)
  if correct_text!='':
    pred_txt.append(pred_ans)
    correct_txt.append(correct_text)
  else:
    correct_txt.append('')
    pred_txt.append('')
  possible_ans.append(possible_answers)
  qs.append(question)
  
  

data=pd.DataFrame({'corpus':document,'question':qs,'possible_answers':possible_ans,'correct_text':correct_txt,'pred_text':pred_txt})
data.head(5)



  9%|▉         | 45/500 [00:01<00:21, 21.35it/s]/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
 22%|██▏       | 108/500 [00:04<00:11, 33.13it/s]/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720

,corpus,question,possible_answers,correct_text,pred_text
0,Email marketing - Wikipedia Email marketing Ju...,which is the most common use of opt-in e-mail ...,[( hide ) This article has multiple issues . P...,A common example of permission marketing is a ...,An exact return on investment can be tracked (...
1,The Mother ( How I Met Your Mother ) - wikiped...,how i.met your mother who is the mother,[Tracy McConnell How I Met Your Mother charact...,"Tracy McConnell , better known as `` The Mothe...","Tracy McConnell , better known as `` The Mothe..."
2,Human fertilization - wikipedia Human fertiliz...,what type of fertilisation takes place in humans,[Part of a series on Human growth and developm...,The process of fertilization involves a sperm ...,The fusion of cell membranes of the secondary ...
3,List of National Football League career quarte...,who had the most wins in the nfl,[NFL lists Current season ( show ) 2017 season...,Active quarterback Tom Brady holds the records...,The following is a list of the top National Fo...
4,Roanoke Colony - wikipedia Roanoke Colony Jump...,what happened to the lost settlement of roanoke,[This article 's lead section does not adequat...,,


In [0]:
score1=f1_score(data['correct_text'],data['pred_text'],average='micro')

In [8]:
print("F1 Score of Confidences is {}".format(score1))

F1 Score of Confidences is 0.556


In [0]:
def jaccard_similarity(query,document):
  intersection=set(query).intersection(set(document))
  union=set(query).union(set(document))
  index=len(intersection)/len(union)
  return index

In [10]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:

correct_txt=list()
possible_ans=list()
qs=list()
pred_txt=list()
for idx in tqdm(range(0,500)):

  correct_text=format_text(remove_html_tags(df.loc[idx,'pred_long_ans']))
  question=format_text(remove_html_tags(df.loc[idx,'question']))
  possible_answers=format_txt(remove_html_tag(eval(df.loc[idx,'candidate_answers'])))

  lemmatizer=WordNetLemmatizer()
  poss_answers=[[lemmatizer.lemmatize(x1,pos="a") for x1 in x.split()] for x in possible_answers]
  ques=[lemmatizer.lemmatize(x,pos="a") for x in question.split()]

  score=list()
  
  for idx, x1 in enumerate(poss_answers):
  
    scr=jaccard_similarity(ques,x1)
    score.append(scr)
  
  pred_ans=possible_answers[np.argmax(score)]

  
  if correct_text!='':
    pred_txt.append(pred_ans)
    correct_txt.append(correct_text)
  else:
    correct_txt.append('')
    pred_txt.append('')
  possible_ans.append(possible_answers)
  qs.append(question)

jaccard_data=pd.DataFrame({'question':qs,'possible_answers':possible_ans,'correct_text':correct_txt,'pred_text':pred_txt})
jaccard_data.head(5)

100%|██████████| 500/500 [00:08<00:00, 56.46it/s]


,question,possible_answers,correct_text,pred_text
0,which is the most common use of opt-in e-mail ...,[( hide ) This article has multiple issues . P...,A common example of permission marketing is a ...,Email marketing is popular with companies for ...
1,how i.met your mother who is the mother,[Tracy McConnell How I Met Your Mother charact...,"Tracy McConnell , better known as `` The Mothe...",The Mother 's real name is not revealed until ...
2,what type of fertilisation takes place in humans,[Part of a series on Human growth and developm...,The process of fertilization involves a sperm ...,The fusion of cell membranes of the secondary ...
3,who had the most wins in the nfl,[NFL lists Current season ( show ) 2017 season...,Active quarterback Tom Brady holds the records...,Players are listed in order of number of wins ...
4,what happened to the lost settlement of roanoke,[This article 's lead section does not adequat...,,


In [0]:
score1=f1_score(jaccard_data['correct_text'],jaccard_data['pred_text'],average='micro')

In [13]:
print("F1 Score of Confidences is {}".format(score1))

F1 Score of Confidences is 0.53


In [14]:
np.sum(jaccard_data['correct_text']==jaccard_data['pred_text'])/len(jaccard_data)

0.53

In [0]:
txt=remove_html_tags(df.loc[0,'text'])

In [0]:
sen=format_text(txt)

In [0]:
from nltk.tokenize import sent_tokenize

In [25]:
sent_tokenize(sen)

['Email marketing - Wikipedia Email marketing Jump to : navigation , search ( hide ) This article has multiple issues .',
 'Please help improve it or discuss these issues on the talk page .',
 '( Learn how and when to remove these template messages ) This article needs additional citations for verification .',
 'Please help improve this article by adding citations to reliable sources .',
 'Unsourced material may be challenged and removed .',
 '( September 2014 ) ( Learn how and when to remove this template message ) This article possibly contains original research .',
 'Please improve it by verifying the claims made and adding inline citations .',
 'Statements consisting only of original research should be removed .',
 '( January 2015 ) ( Learn how and when to remove this template message ) ( Learn how and when to remove this template message ) Part of a series on Internet marketing Search engine optimization Local search engine optimisation Social media marketing Email marketing Refer